# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from tqdm import tnrange, tqdm_notebook
import seaborn as sns
sns.set

Load the environment app to run locally

In [2]:
env = UnityEnvironment(file_name='reacher_single.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 4. Training the Agent

The Agent and replay Buffer are imported from the ddpg_agent.py file.
In order to train multiple agents via MADDPG, the memory is delocated from the individual agents and instantiated globaly.

The Parameters for the memory and agent were adjusted by trial and error and resulted in the implemented values.

Sadly, the multi-agent configuration was unable to use the GPU acceleration, making it impossible for me to run.

In [5]:
# Load the Agent from the ddpg_agent file
from ddpg_agent import Agent, ReplayBuffer
from collections import deque
import torch

import matplotlib.pyplot as plt
%matplotlib inline

# initialize the agent and the environment
#[Agent(state_size=state_size, action_size=action_size, random_seed=0, memory = memory, critic_target = critic_target) for count in range(num_agents)]

# Initialize agent and shared memory (for multi-agent use)
BUFFER_SIZE = int(1e6)  # replay buffer size
BATCH_SIZE = 256        # minibatch size
random_seed = 1

memory ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)
agent = Agent(state_size=state_size, action_size=action_size, random_seed=random_seed, memory = memory)



In [6]:
#training function
def ddpg(n_episodes=150, max_t=1000, PRINT_EVERY=20):
    scores_deque = deque(maxlen=100)
    scores_list = []
    max_score = -np.Inf
    eps = tnrange(1, n_episodes+1)                             # counter for episodes (for time eval)
    for i_episode in eps:
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        agent.reset()
        steps = tnrange(max_t, leave = False)                  # counter for steps (for time eval)
        for t in steps:
            steps.set_description("mean score is %f" % np.mean(scores_deque)) # write last score to counter
            #print(t)
            actions = agent.act(states)                        # calculate next action with actor net
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            agent.step(states, actions, rewards, next_states, dones)
            states = next_states
            scores += rewards
            
            if np.any(dones):
                break
                
        score = np.mean(scores)                                # save scores for evaluation
        scores_deque.append(score)
        scores_list.append(score)
        #steps.set_description("mean score is %f" % np.mean(scores_deque))
        #print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        if i_episode % PRINT_EVERY == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))   
        if np.mean(scores_deque) >= 31:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break
        
    return scores_list

In [ ]:
# train the iteration of the agent and get scores
%time scores = ddpg()

/anaconda3/envs/drlnd/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/envs/drlnd/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Episode 10	Average Score: 0.57


Episode 20	Average Score: 0.77


Episode 30	Average Score: 1.13


Episode 40	Average Score: 1.60


Episode 50	Average Score: 1.92


Episode 60	Average Score: 2.23


Episode 70	Average Score: 2.73


Episode 80	Average Score: 3.33


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
sns.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
env.close()